# Assam District Analysis
> District Analysis for Assam using seirsplus

- toc: true 
- badges: false
- comments: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [0]:
# hide
# !pip install seirsplus
# !pip install geopandas
# !pip install geopatra
# !pip install -U plotly

import warnings
warnings.filterwarnings('ignore')

import contextlib
import io
import json
import random
import sys
from pathlib import Path
from typing import List, Union
import pandas as pd
from branca.colormap import linear
from IPython.display import display
from IPython.utils import io
from ipywidgets import (HTML, FloatLogSlider, FloatSlider, GridBox, HBox,
                        IntSlider, Label, Layout, Output, SelectionSlider,
                        VBox, interactive)
from seirsplus.models import *

In [0]:
assam_df = pd.read_csv("AssamDistrictInfo.csv")

In [0]:
# hide
def get_infections(
    initI: int = 100, initN: int = 10 ** 5, days_N: int = 21, beta: float = 2.4
) -> List[int]:
    model = SEIRSModel(
        beta=beta,
        sigma=1 / 5.2,
        gamma=1 / 12.39,
        initN=initN,
        initI=initI,
        beta_D=0.000,
        mu_D=0.02,
        #         sigma_D=1 / 4.0,
        #         gamma_D=1 / 9.0,
        theta_E=0.0002,
        theta_I=0.002,
        psi_E=0.2,
        psi_I=1.0,
    )
    with io.capture_output() as captured:
        model.run(T=days_N)
    S = model.numS  # time series of S counts
    E = model.numE  # time series of E counts
    I = model.numI  # time series of I counts
    D_E = model.numD_E  # time series of D_E counts
    D_I = model.numD_I  # time series of D_I counts
    R = model.numR  # time series of R counts
    F = model.numF  # time series of F counts
    t = model.tseries  # time values corresponding to the above time series
    return {"detected_exposed": D_E, "detected_infected": D_I, "model": model, "t": t}

In [0]:
# hide
def get_risk_estimates(assam_estimated_df: pd.DataFrame, beta: float) -> Union[List, List]:
    days_N = 21
    atrisk_day14, atrisk_day21 = [], []
    for row in assam_estimated_df[["initI", "initN", "district"]].iterrows():
        initI, initN, district = row[1][0], int(row[1][1]), row[1][2]
        #     print(type(initI), type(initN))
        infection_results = get_infections(
            initI=initI, initN=initN, days_N=days_N, beta=beta
        )
        detected_infected = infection_results["detected_infected"]
        day14 = int(14 * len(detected_infected) / days_N)
        case_count_day14 = int(infection_results["detected_infected"][day14])
        case_count_day21 = int(infection_results["detected_infected"][-1])
        atrisk_day14.append(case_count_day14)
        atrisk_day21.append(case_count_day21)
    return atrisk_day14, atrisk_day21, infection_results

In [0]:
#hide_input
#@title Assumptions
percent_travellers_infected = 1e-1 #@param {type:"slider", min:0, max:1e-1, step:0.001}
unknown_to_known_travelers = 50 #@param {type:"slider", min:5, max:50, step:1}
beta = 5 #@param {type:"slider", min:0.5, max:5.0, step:0.1}

In [0]:
#hide
def estimate(percent_travellers_infected: float, 
            unknown_to_known_travelers:int,
            beta:float, 
            assam_df:pd.DataFrame):
    
    assam_estimated_df = assam_df.copy()
    assam_estimated_df["initI"] = (
        percent_travellers_infected * unknown_to_known_travelers * assam_df["Inflow"]
    ).astype(int)
    assam_estimated_df["initN"] = assam_estimated_df["Population"].astype(int)
    
    assam_estimated_df[f"day14"] , assam_estimated_df[f"day21"], infection_results = get_risk_estimates(assam_estimated_df, beta=beta)
    return assam_estimated_df

In [0]:
#hide
assam_estimated_df = estimate(percent_travellers_infected, 
              unknown_to_known_travelers,
              beta, 
              assam_df)

In [0]:
# hide
assam_estimated_df["AtRisk"] = assam_estimated_df["day21"]

In [0]:
# hide
assam_estimated_df["text"] = (
    "District : "
    + assam_estimated_df["district"].astype(str)
    + "<br>"
    + "Population: "
    + assam_estimated_df["Population"].astype(str)
    + "<br>"
    + "Day14 Exposed Estimate : "
    + assam_estimated_df["day14"].astype(str)
    + "<br>"
    + "Day21 Exposed Estimate : "
    + assam_estimated_df["day21"].astype(str)
    + "<br>"
    + "Travellers: "
    + assam_estimated_df["Inflow"].astype(str)
    
)

assam_estimated_df.head()

In [0]:
# hide_input
assam_estimated_df[["district", "AtRisk"]]

In [0]:
# hide
from urllib.request import urlopen
import json

with urlopen(
    "https://raw.githubusercontent.com/NirantK/coronaIndia/voila/voila_notebook/Assam/assam.json"
) as response:
    KA = json.load(response)

In [0]:
# hide
import plotly.graph_objects as go


fig = go.Figure(
    data=go.Choropleth(
        locations=assam_estimated_df["district"],
        z=assam_estimated_df["day21"].astype(float),
        colorscale="Blues",
        autocolorscale=False,
        reversescale=False,
        marker_line_color="darkgray",
        marker_line_width=0.5,
        geojson=KA,
        featureidkey="properties.district",
        colorbar_title="At Risk",
        text=assam_estimated_df["text"],
        hoverinfo="text"
    )
)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title_text="District Analysis Assam: Map")

In [0]:
# hide_input
fig.show()

# Infections Plot

In [0]:
# hide_input
beta = 2.4
_, _, infection_results = get_risk_estimates(assam_estimated_df, beta=beta)
infection_results["model"].figure_infections(
            plot_percentages=False,
            plot_D_E=False,
            title = "Plausible Forecast",
#             title=f"Plausible Scenario for {district}. Population > {initN / 10**6:.2f} Million",
            side_title="This model is intended to help make fast decisions, not predict the future.",
            plot_E=False,
            plot_I=False,
            color_D_I="tab:blue",
)